### Experiments with zero-shot classification techniques to classify manual test cases (i.e., textual descriptions of test cases) into the game features that they cover.

We experiment with the following models/techniques and their combinations:

* BartLargeMNLI - [facebook/bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli)
* CrossEncoderNLI - [cross-encoder/nli-distilroberta-base](https://huggingface.co/cross-encoder/nli-distilroberta-base)
* LatentEmb - [latent-embeddings](https://joeddav.github.io/blog/2020/05/29/ZSL.html)

In [9]:
# Import necessary libraries
import os
import re
import time
import string
import pandas as pd
import numpy as np
from statistics import median, mean
import pathlib
from pathlib import Path
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
import torch
import torch.nn.functional as F
from nltk.tokenize import word_tokenize, TweetTokenizer
import nltk 
from nltk.corpus import stopwords
import gensim.downloader as api
from gensim.models import Word2Vec, Phrases, KeyedVectors
import fasttext
from scipy import spatial
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import multilabel_confusion_matrix, precision_recall_fscore_support
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib
import warnings
from IPython.display import display, HTML
warnings.filterwarnings("ignore")

In [ ]:
# Notebook configurations
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(HTML("<style>.container { width:100% !important; }</style>"))

In [15]:
# Import modules with different classification methods
from zero_shot_nli import run_zero_shot_nli
from zero_shot_nli_metrics_per_class import run_zero_shot_nli_metrics_per_class
from zero_shot_latent_embedding import run_zero_shot_latent_emb
from baseline import run_baseline
import utils

---

### Load and pre-process labeled data

In [ ]:
# Load labeled data
labeled_test_cases_df = utils.read_data()
labeled_test_cases_df.head()

In [ ]:
# Pre-process data
(test_case_name_df, test_case_name_obj_df) = utils.preprocess_data(labeled_test_cases_df)

In [ ]:
# Get list of unique labels (game features)
unique_labels = []
for index,row in test_case_name_df.iterrows():
    labels = row['labels']
    for lab in labels:
        if lab not in unique_labels:
            unique_labels.append(lab)

In [ ]:
# Dict with counter of unique labels
unique_labels_count = dict.fromkeys(unique_labels,0)
for index,row in test_case_name_df.iterrows():
    labels = row['labels']
    for lab in labels:
        unique_labels_count[lab] += 1

In [ ]:
# Avg number of unique labels
mean_label_counter = mean(list(unique_labels_count.values()))
print("There are on average {count} unique labels.".format(count=mean_label_counter))

In [ ]:
# Load labels (game features)
candidate_label_file = "INSERT_DIR_OF_LIST_OF_GAME_FEATURES"
candidate_labels = candidate_label_file.read().splitlines()
print("There are {count} candidate labels.".format(count=len(candidate_labels)))

In [ ]:
# Replace dash by space in candidate labels with more than one word (achieves better performance)
candidate_labels_mod = []
for elem in candidate_labels:
    res = ' '.join(elem.split('-'))
    candidate_labels_mod.append(res)

In [12]:
# Set Mlflow experiment dir
experiment_dir = "INSERT_DIR_TO_RECORD_EXPERIMENTS_WITH_MLFLOW"

---

### Baseline

In [ ]:
# Define name and description of experiment
experiment_name = "Baseline experiment - Test case name and objective"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate keyword-based approach to classify test cases (with test case name and objective).")

In [ ]:
# Test cases represented by name
run_name = "Test case name"
run_baseline(test_case_name_df, candidate_labels, candidate_labels_mod, experiment_name, run_name, experiment_dir)

In [ ]:
# Test cases represented by name + objective
run_name = "Test case name + objective"
run_baseline(test_case_name_obj_df, candidate_labels, candidate_labels_mod, experiment_name, run_name, experiment_dir)

### Experiments with individual zero-shot techniques

#### BartLargeMNLI - [facebook/bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli)

In [ ]:
# Define name and description of experiment
experiment_name = "BartLargeMNLI - Test case name and objective"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate BartLargeMNLI to classify test cases (with test case name and objective).")

In [ ]:
# Load zero-shot classifier from the HF pipeline - set device=0 to use GPU for faster inference
zero_shot_nli_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

In [ ]:
# Run classifier - considering test case as test case name
run_name = "Test case name"
run_zero_shot_nli(zero_shot_nli_classifier, candidate_labels, test_case_name_df, experiment_name, run_name, experiment_dir)

In [ ]:
# Run classifier - considering test case as test case name + test case objective
run_name = "Test case name + objective"
run_zero_shot_nli(zero_shot_nli_classifier, candidate_labels, test_case_name_obj_df, experiment_name, run_name, experiment_dir)

#### CrossEncoderNLI - [cross-encoder/nli-distilroberta-base](https://huggingface.co/cross-encoder/nli-distilroberta-base)

In [ ]:
# Define name and description of experiment
experiment_name = "CrossEncoderNLI - Test case name and objective"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate CrossEncoderNLI to classify test cases (with test case name and objective).")

In [ ]:
# Load zero-shot classifier from the HF pipeline - set device=0 to use GPU for faster inference
zero_shot_nli_cross_enc_classifier = pipeline("zero-shot-classification", model='cross-encoder/nli-distilroberta-base', device=0)

In [ ]:
# Run classifier - considering test case as test case name
run_name = "Test case name"
run_zero_shot_nli(zero_shot_nli_cross_enc_classifier, candidate_labels, test_case_name_df, experiment_name, run_name, experiment_dir)

In [ ]:
# Run classifier - considering test case as test case name + test case objective
run_name = "Test case name + objective"
run_zero_shot_nli(zero_shot_nli_cross_enc_classifier, candidate_labels, test_case_name_obj_df, experiment_name, run_name, experiment_dir)

#### LatentEmb - [latent-embeddings](https://joeddav.github.io/blog/2020/05/29/ZSL.html)

We experiment with Wor2vec, Fasttext, and Glove embedding models together with the SBERT sentence embedding model

In [ ]:
# Define name and description of experiment
experiment_name = "Zero-shot Latent embeddings - Word embbeding models"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate different word embedding models for zero-shot with latent embeddings approach")

In [ ]:
sbert_name = 'sentence-t5-large'
sbert_model = SentenceTransformer(sbert_name, device='cuda')

In [ ]:
def ordinary_least_squares_lr(
    X: torch.Tensor, Y: torch.Tensor, alpha: float = 0) -> torch.Tensor:
    """Computes ordinary least squares
    For more information on the derivation of the closed-form expression,
    check it the Wikipedia page here:
    https://en.wikipedia.org/wiki/Ordinary_least_squares#Matrix/vector_formulation
    In brief: we find a matrix, w, that transforms X to Y according to:
    Y = Xw
    (X.T X)^-1 X.T Y = [(X.T X)^-1 X.T X]w
    w = (X.T X + alpha*I)^-1 X.T Y
    where I is the identity matrix and alpha is the amount of regularization.
    alpha = 0 is equivalent to OLS (ordinary least squares)
    alpha >= 0 is ridge regression / l2 regularization
    """
    X_norm = F.normalize(X, p=2, dim=1)
    Y_norm = F.normalize(Y, p=2, dim=1)
    I = torch.eye(X_norm.shape[1])

    inner = torch.matmul(X_norm.T, X_norm) + alpha * I
    # Z = torch.linalg.inv(inner)
    Z = torch.inverse(inner)
    Z = torch.matmul(Z, X_norm.T)
    w = torch.matmul(Z, Y_norm)
    return w

##### Zero-shot Latent embeddings - Word2Vec

In [ ]:
# Load word2vec pre-trained model
w2v_model = api.load('word2vec-google-news-300')

In [ ]:
topk_words_pretrained = w2v_model.index_to_key[:20000]
print("Len of topk word vector", len(topk_words_pretrained))

# Remove stopwords
topk_words_pretrained = [x for x in topk_words_pretrained if x not in stopwords.words('english')]

# Remove punctuations
topk_words_pretrained = [x for x in topk_words_pretrained if ( (x not in string.punctuation) or (x[0] not in string.punctuation) )]

# Remove single letters/digits
topk_words_pretrained = [x for x in topk_words_pretrained if len(x) > 1]

# Remove any remaining number
topk_words_pretrained = [x for x in topk_words_pretrained if not x.isdigit()]

print("Len of topk word vector after filtering", len(topk_words_pretrained))

In [ ]:
# Get w2v embeddings
w2v_emb_vectors = []
for word in topk_words_pretrained:
    w2v_emb_vectors.append(w2v_model.get_vector(word))
    
w2v_emb_vectors = np.array(w2v_emb_vectors)
w2v_emb_vectors = torch.tensor(w2v_emb_vectors)
print("Len of w2v embedding vector list", len(w2v_emb_vectors))

In [ ]:
# Get SBERT embeddings for the same set of words
sbert_emb_vectors = []
for word in topk_words_pretrained:
    sbert_emb_vectors.append(sbert_model.encode(word))
    
sbert_emb_vectors = np.array(sbert_emb_vectors)
sbert_emb_vectors = torch.tensor(sbert_emb_vectors)
print("Len of sbert embedding vector list", len(sbert_emb_vectors))

In [ ]:
# Compute transfer matrix
transfer_matrix = ordinary_least_squares_lr(sbert_emb_vectors, w2v_emb_vectors, alpha=0)
print(transfer_matrix.shape)

In [ ]:
# Get embeddings of candidate labels
candidate_label_embeddings = sbert_model.encode(candidate_labels_mod)
print(candidate_label_embeddings.shape)

# Covert to tensor
candidate_label_embeddings = torch.tensor(candidate_label_embeddings)
print(candidate_label_embeddings.shape)

# Apply linear transformation
candidate_label_embeddings_transformed = torch.mm(candidate_label_embeddings, transfer_matrix)
print(candidate_label_embeddings_transformed.shape)

In [ ]:
# Run classifier - considering test case as test case name
run_name = 'Test case name - ' + 'Word2Vec + ' + sbert_name
run_zero_shot_latent_emb(test_case_name_df, candidate_labels_mod, candidate_label_embeddings_transformed,
                         sbert_model, transfer_matrix, experiment_name, run_name, experiment_dir)

In [ ]:
# Run classifier - considering test case as test case name + test case objective
run_name = 'Test case name + objective - ' + 'Word2Vec + ' + sbert_name
run_zero_shot_latent_emb(test_case_name_obj_df, candidate_labels_mod, candidate_label_embeddings_transformed,
                         sbert_model, transfer_matrix, experiment_name, run_name, experiment_dir)

##### Zero-shot Latent embeddings - Fasttext

In [ ]:
# fasttext.util.download_model('en', if_exists='ignore')  # English
fasttext_model = fasttext.load_model("INSERT_PATH_OF_MODEL")

In [ ]:
topk_words_pretrained = fasttext_model.get_words()[:20000]
print("Len of topk word vector", len(topk_words_pretrained))

# Remove stopwords
topk_words_pretrained = [x for x in topk_words_pretrained if x not in stopwords.words('english')]

# Remove punctuations
topk_words_pretrained = [x for x in topk_words_pretrained if ( (x not in string.punctuation) or (x[0] not in string.punctuation) )]

# Remove single letters/digits
topk_words_pretrained = [x for x in topk_words_pretrained if len(x) > 1]

# Remove any remaining number
topk_words_pretrained = [x for x in topk_words_pretrained if not x.isdigit()]

print("Len of topk word vector after filtering", len(topk_words_pretrained))

In [ ]:
# Get fasttext embeddings
ft_emb_vectors = []
for word in topk_words_pretrained:
    ft_emb_vectors.append(fasttext_model.get_word_vector(word))
    
ft_emb_vectors = np.array(ft_emb_vectors)
ft_emb_vectors = torch.tensor(ft_emb_vectors)
print("Len of fasttext embedding vector list", len(ft_emb_vectors))

In [ ]:
# Get SBERT embeddings for the same set of words
sbert_emb_vectors = []
for word in topk_words_pretrained:
    sbert_emb_vectors.append(sbert_model.encode(word))
    
sbert_emb_vectors = np.array(sbert_emb_vectors)
sbert_emb_vectors = torch.tensor(sbert_emb_vectors)
print("Len of sbert embedding vector list", len(sbert_emb_vectors))

In [ ]:
# Compute transfer matrix
transfer_matrix = ordinary_least_squares_lr(sbert_emb_vectors, ft_emb_vectors, alpha=0)
print(transfer_matrix.shape)

In [ ]:
# Get embeddings of candidate labels
candidate_label_embeddings = sbert_model.encode(candidate_labels_mod)
print(candidate_label_embeddings.shape)

# Covert to tensor
candidate_label_embeddings = torch.tensor(candidate_label_embeddings)
print(candidate_label_embeddings.shape)

# Apply linear transformation
candidate_label_embeddings_transformed = torch.mm(candidate_label_embeddings, transfer_matrix)
print(candidate_label_embeddings_transformed.shape)

In [ ]:
# Run classifier - considering test case as test case name
run_name = 'Test case name - ' + 'Fasttext + ' + sbert_name
run_zero_shot_latent_emb(test_case_name_df, candidate_labels_mod, candidate_label_embeddings_transformed,
                         sbert_model, transfer_matrix, experiment_name, run_name, experiment_dir)

In [ ]:
# Run classifier - considering test case as test case name + test case objective
run_name = 'Test case name + objective - ' + 'Fasttext + ' + sbert_name
run_zero_shot_latent_emb(test_case_name_obj_df, candidate_labels_mod, candidate_label_embeddings_transformed,
                         sbert_model, transfer_matrix, experiment_name, run_name, experiment_dir)

##### Zero-shot Latent embeddings - Glove

In [ ]:
# Create dictionary with word embeddings from Glove
embeddings_index = {}
f = open('INSERT_PATH_OF_MODEL','r',encoding='utf-8')
for line in f:
    values = line.split(' ')
    word = values[0]
    vector = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = vector
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# Get glove embeddings (using the same 'topk_words_pretrained' as before since we cannot get word frequency from glove)
glove_emb_vectors = []
sbert_emb_vectors = []

for word in topk_words_pretrained:
    try:
        glove_emb_vectors.append(embeddings_index[word])
        sbert_emb_vectors.append(sbert_model.encode(word))
    except:
        continue

glove_emb_vectors = np.array(glove_emb_vectors)
glove_emb_vectors = torch.tensor(glove_emb_vectors)

# Convert from float64 (double) to float
glove_emb_vectors = glove_emb_vectors.type(torch.float32) 
print("Len of glove embedding vector list", len(glove_emb_vectors))

sbert_emb_vectors = np.array(sbert_emb_vectors)
sbert_emb_vectors = torch.tensor(sbert_emb_vectors)
print("Len of sbert embedding vector list", len(sbert_emb_vectors))

In [ ]:
# Compute transfer matrix
transfer_matrix = ordinary_least_squares_lr(sbert_emb_vectors, glove_emb_vectors, alpha=0)
print(transfer_matrix.shape)

In [ ]:
# Get embeddings of candidate labels
candidate_label_embeddings = sbert_model.encode(candidate_labels_mod)
print(candidate_label_embeddings.shape)

# Covert to tensor
candidate_label_embeddings = torch.tensor(candidate_label_embeddings)
print(candidate_label_embeddings.shape)

# Apply linear transformation
candidate_label_embeddings_transformed = torch.mm(candidate_label_embeddings, transfer_matrix)
print(candidate_label_embeddings_transformed.shape)

In [ ]:
# Run classifier - considering test case as test case name
run_name = 'Test case name - ' + 'Glove + ' + sbert_name
run_zero_shot_latent_emb(test_case_name_df, candidate_labels_mod, candidate_label_embeddings_transformed,
                         sbert_model, transfer_matrix, experiment_name, run_name, experiment_dir)

In [ ]:
# Run classifier - considering test case as test case name + test case objective
run_name = 'Test case name + objective - ' + 'Glove + ' + sbert_name
run_zero_shot_latent_emb(test_case_name_obj_df, candidate_labels_mod, candidate_label_embeddings_transformed,
                         sbert_model, transfer_matrix, experiment_name, run_name, experiment_dir)

### Experiments with **ensembles** of individual zero-shot techniques

#### Load individual models

In [ ]:
# Load models for latent embeddings approach
sbert_name = 'sentence-t5-large'
sbert_model = SentenceTransformer(sbert_name, device='cuda')

# Compute transfer matrix with best word embedding model (word2vec)
transfer_matrix = ordinary_least_squares_lr(sbert_emb_vectors, w2v_emb_vectors, alpha=0)
print(transfer_matrix.shape)

# Compute candidate label embeddings with linear transformation
candidate_label_embeddings_mod = sbert_model.encode(candidate_labels_mod)
candidate_label_embeddings_mod = torch.tensor(candidate_label_embeddings_mod)
candidate_label_embeddings_mod_transformed = torch.mm(candidate_label_embeddings_mod, transfer_matrix)
print(candidate_label_embeddings_mod_transformed.shape)

In [ ]:
# Load models for NLI approach
zero_shot_nli_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

In [ ]:
# Load models for NLI cross-encoder approach
zero_shot_nli_cross_en_classifier = pipeline("zero-shot-classification", model='cross-encoder/nli-distilroberta-base', device=0)

In [ ]:
# Instantiate and fit multi-label binarizer (one-hot encoding for multi-label)
mlb  = MultiLabelBinarizer()
mlb.fit([candidate_labels])

#### Get classifications with BartLargeMNLI, CrossEncoderNLI, and SBERT (to avoid redundant computations)

In [ ]:
# Get classification with BartLargeMNLI - test case name
analyzed_key = []
classif_res_list_BartLargeMNLI_name = []

for index,row in test_case_name_df.iterrows():
    # Get test case name
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Embed test description
    classification_res = zero_shot_nli_classifier(test_description, candidate_labels, multi_label=True)
    classif_res_list_BartLargeMNLI_name.append(classification_res)
    analyzed_key.append(test_key)

In [ ]:
# Get classification with BartLargeMNLI - test case name + objective
analyzed_key = []
classif_res_list_BartLargeMNLI_name_obj = []

for index,row in test_case_name_obj_df.iterrows():
    # Get test case name
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Embed test description
    classification_res = zero_shot_nli_classifier(test_description, candidate_labels, multi_label=True)
    classif_res_list_BartLargeMNLI_name_obj.append(classification_res)
    analyzed_key.append(test_key)

In [ ]:
# Get classification with CrossEncoderNLI - test case name
analyzed_key = []
classif_res_list_CrossEncoderNLI_name = []

for index,row in test_case_name_df.iterrows():
    # Get test case name
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Embed test description
    classification_res = zero_shot_nli_cross_en_classifier(test_description, candidate_labels, multi_label=True)
    classif_res_list_CrossEncoderNLI_name.append(classification_res)
    analyzed_key.append(test_key)

In [ ]:
# Get classification with CrossEncoderNLI - test case name + objective
analyzed_key = []
classif_res_list_CrossEncoderNLI_name_obj = []

for index,row in test_case_name_obj_df.iterrows():
    # Get test case name
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Embed test description
    classification_res = zero_shot_nli_cross_en_classifier(test_description, candidate_labels, multi_label=True)
    classif_res_list_CrossEncoderNLI_name_obj.append(classification_res)
    analyzed_key.append(test_key)

In [ ]:
# Get classification from SBERT - test case name
analyzed_key = []
classif_res_list_SBERT_name = []

for index,row in test_case_name_df.iterrows():
    # Add test case name
    test_key = row['id']
    test_description = row['description']

    if test_key in analyzed_key:
        continue

    # Embed test description
    sentence_embedding = sbert_model.encode(test_description)

    # Apply linear transformation to sentence embedding
    sentence_embedding = torch.tensor(sentence_embedding)
    sentence_embedding = torch.reshape(sentence_embedding, (1, len(sentence_embedding)))
    sentence_embedding_transformed = torch.mm(sentence_embedding, transfer_matrix)
                
    classif_res_list_SBERT_name.append(sentence_embedding_transformed)
    analyzed_key.append(test_key)

In [ ]:
# Get classification from SBERT - test case name + objective
analyzed_key = []
classif_res_list_SBERT_name_obj = []

for index,row in test_case_name_obj_df.iterrows():
    # Add test case name
    test_key = row['id']
    test_description = row['description']

    if test_key in analyzed_key:
        continue

    # Embed test description
    sentence_embedding = sbert_model.encode(test_description)

    # Apply linear transformation to sentence embedding
    sentence_embedding = torch.tensor(sentence_embedding)
    sentence_embedding = torch.reshape(sentence_embedding, (1, len(sentence_embedding)))
    sentence_embedding_transformed = torch.mm(sentence_embedding, transfer_matrix)
                
    classif_res_list_SBERT_name_obj.append(sentence_embedding_transformed)
    analyzed_key.append(test_key)

#### EnsMajorVoting - Ensemble with majority voting

In [ ]:
# Define name and description of experiment
experiment_name = "Ensemble with majority voting - Test case name and objective"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate how the ensemble of all the zero-shot techniques using majority voting performs for test case name and objective.")

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    # Count number of models that assigned each label to current test case
    count_label = { i : 0 for i in candidate_labels }
    for label in candidate_labels:
        if label in labels_nli:
            count_label[label] += count_label[label] + 1

        if label in labels_nli_cross_enc:
            count_label[label] += count_label[label] + 1

        if label in labels_emb:
            count_label[label] += count_label[label] + 1

    # Select labels assigned by at least 2 models
    intersection_labels = set()
    for label in count_label:
        if count_label[label] >= 2:
            intersection_labels.add(label)

    correct_labels = test_case_name_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name + objective"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_obj_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name_obj[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]

    
    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    # Count number of models that assigned each label to current test case
    count_label = { i : 0 for i in candidate_labels }
    for label in candidate_labels:
        if label in labels_nli:
            count_label[label] += count_label[label] + 1

        if label in labels_nli_cross_enc:
            count_label[label] += count_label[label] + 1

        if label in labels_emb:
            count_label[label] += count_label[label] + 1

    # Select labels assigned by at least 2 models
    intersection_labels = set()
    for label in count_label:
        if count_label[label] >= 2:
            intersection_labels.add(label)

    correct_labels = test_case_name_obj_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 

#### EnsFullInters - Ensemble with full intersection

In [ ]:
# Define name and description of experiment
experiment_name = "Ensemble with full intersection - Test case name and objective"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate how the ensemble of full intersection of all the zero-shot techniques performs for test case name and objective.")

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    intersection_labels_nli = labels_nli.intersection(labels_nli_cross_enc)
    intersection_labels = intersection_labels_nli.intersection(labels_emb)

    correct_labels = test_case_name_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name + objective"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_obj_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name_obj[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]

    
    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    intersection_labels_nli = labels_nli.intersection(labels_nli_cross_enc)
    intersection_labels = intersection_labels_nli.intersection(labels_emb)

    correct_labels = test_case_name_obj_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 

#### EnsBackOffTwo - Ensemble with back-off using top-2 models

In [ ]:
# Define name and description of experiment
experiment_name = "Ensemble with back-off using top-2 models - Test case name and objective"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate how the ensemble with back-off using top-2 models performs for test case name and objective.")

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    intersection_labels_nli = labels_nli.intersection(labels_nli_cross_enc)
    intersection_labels = intersection_labels_nli.intersection(labels_emb)

    if len(intersection_labels) == 0:
        intersection_labels = labels_emb.intersection(labels_nli)

        if len(intersection_labels) == 0:
            intersection_labels = labels_emb.intersection(labels_nli_cross_enc)

    correct_labels = test_case_name_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name + objective"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_obj_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name_obj[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]

    
    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    intersection_labels_nli = labels_nli.intersection(labels_nli_cross_enc)
    intersection_labels = intersection_labels_nli.intersection(labels_emb)

    if len(intersection_labels) == 0:
        intersection_labels = labels_emb.intersection(labels_nli)

        if len(intersection_labels) == 0:
            intersection_labels = labels_emb.intersection(labels_nli_cross_enc)

    correct_labels = test_case_name_obj_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 

#### EnsBackOffComplete - Ensemble with back-off using all models

In [ ]:
# Define name and description of experiment
experiment_name = "Ensemble with back-off using all the models - Test case name and objective"
experiment_active = mlflow.set_experiment(experiment_name)
experiment_id = experiment_active.experiment_id
MlflowClient().set_experiment_tag(experiment_id, 
     "mlflow.note.content","Evaluate how the ensemble with back-off using all the models performs for test case name and objective.")

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    intersection_labels_nli = labels_nli.intersection(labels_nli_cross_enc)
    intersection_labels = intersection_labels_nli.intersection(labels_emb)

    if len(intersection_labels) == 0:
        intersection_labels = labels_emb.intersection(labels_nli)

        if len(intersection_labels) == 0:
            intersection_labels = labels_emb.intersection(labels_nli_cross_enc)

            if len(intersection_labels) == 0:
                intersection_labels = labels_nli.intersection(labels_nli_cross_enc)

    correct_labels = test_case_name_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 

In [ ]:
mlflow.set_tracking_uri(experiment_dir)
experiment_active = mlflow.set_experiment(experiment_name)
run_name = "Test case name + objective"

In [ ]:
# Use optimal similarity thresholds as found in the experiments with individual models
optimal_sim_thresh_embedding = 0.2
optimal_sim_thresh_nli = 0.9
optimal_sim_thresh_nli_cross_enc = 0.6

# Dataframe to store embedding predictions
classified_test_cases_embedding = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store NLI predictions
classified_test_cases_nli = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

# Dataframe to store cross-encoder NLI predictions
classified_test_cases_nli_cross_enc = pd.DataFrame(columns=['test_key', 'test_name', 'labels'])

index_add = 0
analyzed_key = []

for index in range(len(test_case_name_obj_df)):
    row = test_case_name_df.iloc[index,:]     
    test_key = row['id']
    test_description = row['description']
    correct_labels = row['labels']

    if test_key in analyzed_key:
        continue

    # Get predictions with latent embeddings
    sentence_embedding_transformed = classif_res_list_SBERT_name_obj[index]

    labels_to_include = set()
    
    # Iterate through candidate labels and the modified candidate label embeddings
    for label, label_embedding in zip(candidate_labels, candidate_label_embeddings_mod_transformed):
        cos_sim = util.cos_sim(sentence_embedding_transformed, label_embedding)
        if cos_sim > optimal_sim_thresh_embedding:
            labels_to_include.add(label)

    classified_test_cases_embedding.loc[index_add] = [test_key, test_description, labels_to_include]

    
    # Get predictions from NLI model
    classif_res = classif_res_list_BartLargeMNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli.loc[index_add] = [test_key, test_description, labels_to_include]


    # Get predictions from cross-encoder NLI model
    classif_res = classif_res_list_CrossEncoderNLI_name_obj[index]

    # Check we are looking at the correct instance
    if test_description != classif_res['sequence']:
        print("Error!")
        break

    predicted_labels = classif_res['labels']
    prediction_scores = classif_res['scores']

    # Iterate through predictions (descendent order by score)
    labels_to_include = set()
    scores_to_include = []
    for label,score in zip(predicted_labels, prediction_scores):
        if score < optimal_sim_thresh_nli_cross_enc:
            break
        else:
            labels_to_include.add(label)
            scores_to_include.append(score) 

    classified_test_cases_nli_cross_enc.loc[index_add] = [test_key, test_description, labels_to_include]
    index_add += 1
    analyzed_key.append(test_key)

# Dataframe to store combination of predictions
final_classified_test_cases = pd.DataFrame(columns=['test_key', 'test_name', 'correct_labels', 'pred_labels', 'encoded_correct_labels', 'encoded_pred_labels'])
index_add = 0

analyzed_key = []

for index in range(len(classified_test_cases_nli)):
    row = classified_test_cases_nli.iloc[index,:]
    test_key_nli = row['test_key']
    test_name_nli = row['test_name']
    labels_nli = row['labels']

    if test_key_nli in analyzed_key:
        continue

    row = classified_test_cases_nli_cross_enc.iloc[index,:]
    test_key_nli_cross_enc = row['test_key']
    test_name_nli_cross_enc = row['test_name']
    labels_nli_cross_enc = row['labels']


    row_emb = classified_test_cases_embedding.iloc[index,:]
    test_key_emb = row_emb['test_key']
    test_name_emb = row_emb['test_name']
    labels_emb = row_emb['labels']

    if (test_key_nli != test_key_nli_cross_enc) or (test_key_nli != test_key_emb):
        print("Error occurred!")
        break

    intersection_labels_nli = labels_nli.intersection(labels_nli_cross_enc)
    intersection_labels = intersection_labels_nli.intersection(labels_emb)

    if len(intersection_labels) == 0:
        intersection_labels = labels_emb.intersection(labels_nli)

        if len(intersection_labels) == 0:
            intersection_labels = labels_emb.intersection(labels_nli_cross_enc)

            if len(intersection_labels) == 0:
                intersection_labels = labels_nli.intersection(labels_nli_cross_enc)

    correct_labels = test_case_name_obj_df.iloc[index,3]

    # Encoded labels
    encoded_correct_labels = mlb.transform([correct_labels])
    encoded_labels_to_include = mlb.transform([intersection_labels])     

    final_classified_test_cases.loc[index_add] = [test_key_nli, test_name_nli, correct_labels, intersection_labels, encoded_correct_labels, encoded_labels_to_include]
    index_add += 1

    analyzed_key.append(test_key_nli)


with mlflow.start_run(experiment_id=experiment_active.experiment_id, run_name=run_name):
    y_true = []
    y_pred = []
    for index,row in final_classified_test_cases.iterrows():
        label_encoded = row['encoded_correct_labels']
        predicted_label_encoded = row['encoded_pred_labels']
        y_true.append(label_encoded[0])
        y_pred.append(predicted_label_encoded[0])

    # Get metrics
    metrics = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    precision = metrics[0]
    recall = metrics[1]
    fscore = metrics[2]

    end_time = time.time()
    duration = end_time - start_time
    duration = "{0:.2f}".format(duration)

    mlflow.log_param("Embedding similarity threshold", optimal_sim_thresh_embedding)
    mlflow.log_param("NLI similarity threshold", optimal_sim_thresh_nli)
    mlflow.log_param("NLI cross-encoder similarity threshold", optimal_sim_thresh_nli_cross_enc)
    mlflow.log_param("Execution time", duration)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("fscore", fscore) 